In [ ]:
Парсер частных школ РФ

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re

In [2]:
URL = 'https://www.orgpage.ru/rossiya/%D1%87%D0%B0%D1%81%D1%82%D0%BD%D1%8B%D0%B5_%D1%88%D0%BA%D0%BE%D0%BB%D1%8B/'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0', 'accept': '*/*'}
FILE = 'schools.csv'

In [3]:
def get_html(url, params=None):
    request = requests.get(url, headers=HEADERS, params=params)
    return request

In [4]:
def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find_all('a', class_='gradient')[1:-1]
    if pagination:
        return int(pagination[-1].get_text())
    else:
        return 1

In [5]:
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='object-item similar-item')
    schools = []
    for item in items:
        schools.append({
            'title': item.find('div', class_='similar-item__title').get_text(strip=True).lstrip('0123456789.'),
            'description': item.find('div', class_='similar-item__description').get_text(strip=True),
            'phone': re.sub("[-|+|)|(|-| ]","",item.find('div', class_='phone').get_text(strip=True)),
            'site': item.find('div', class_='similar-item__phone').get_text(strip=True).lstrip('+ )(-0123456789.'),
            'region': item.find('div', class_='similar-item__adrss-item').get_text(strip=True).split(",")[0],
            'address': item.find('div', class_='similar-item__adrss-item').get_text(strip=True),
        })
    return schools


In [6]:
def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Школа', 'Описание', 'Телефон','Сайт','Регион','Адрес'])
        for item in items:
            writer.writerow([item['title'], item['description'], item['phone'],
                             item['site'], item['region'], item['address']])

In [7]:
def parse():
    html = get_html(URL)
    if html.status_code == 200:
        schools = []
        pages_count = get_pages_count(html.text)
        for page in range(1, pages_count + 1):
            print(f'Выполнено {page} из {pages_count}...')
            html = get_html(URL, params={'page': page})
            schools.extend(get_content(html.text))
        save_file(schools, FILE)
        print(f'Получено {len(schools)} строк')
        os.startfile(FILE)
    else:
        print('Error')


parse()

Парсинг страницы 1 из 10...
Парсинг страницы 2 из 10...
Парсинг страницы 3 из 10...
Парсинг страницы 4 из 10...
Парсинг страницы 5 из 10...
Парсинг страницы 6 из 10...
Парсинг страницы 7 из 10...
Парсинг страницы 8 из 10...
Парсинг страницы 9 из 10...
Парсинг страницы 10 из 10...
Получено 270 школ
